In [5]:
#import pandas as pd
!wget http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
!unzip dataset_tsmc2014.zip

--2024-05-21 12:28:10--  http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
Resolving www-public.tem-tsp.eu (www-public.tem-tsp.eu)... 157.159.10.107, 2001:660:3203:100:1:0:80:107
Connecting to www-public.tem-tsp.eu (www-public.tem-tsp.eu)|157.159.10.107|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25546284 (24M) [application/zip]
Saving to: ‘dataset_tsmc2014.zip’

dataset_tsmc2014.zi 100%[===================>]  24.36M  5.61MB/s    in 6.2s    

2024-05-21 12:28:17 (3.95 MB/s) - ‘dataset_tsmc2014.zip’ saved [25546284/25546284]

Archive:  dataset_tsmc2014.zip
   creating: dataset_tsmc2014/
  inflating: dataset_tsmc2014/dataset_TSMC2014_NYC.txt  
  inflating: dataset_tsmc2014/dataset_TSMC2014_readme.txt  
  inflating: dataset_tsmc2014/dataset_TSMC2014_TKY.txt  


In [6]:
import pandas as pd
columns = ['User ID',
 'Venue ID',
 'Venue category ID',
 'Venue category name',
 'Latitude',
 'Longitude',
 'Timezone',
 'UTC time']

In [7]:
NUMCHECKIN = 30000

df = pd.read_csv('dataset_tsmc2014/dataset_TSMC2014_NYC.txt', sep='\t', encoding='latin-1', names=columns)
df = df.sample(NUMCHECKIN)
print(df)

        User ID                  Venue ID         Venue category ID  \
42651       667  4e3cb594fa76455375af1684  4bf58dd8d48988d103941735   
63402       424  4a32c3f9f964a5208c9a1fe3  4bf58dd8d48988d1df941735   
211582      463  4fbc40f5e4b0cc7b013f2642  4bf58dd8d48988d120941735   
171125       25  4bb494c2f038a5935b29daf0  4bf58dd8d48988d124941735   
164508      570  45938647f964a52045401fe3  4bf58dd8d48988d175941735   
58374       178  4bfade10ab180f475e11b3ce  4bf58dd8d48988d122951735   
182287      806  49cae9c7f964a520ef581fe3  4bf58dd8d48988d121941735   
86066       289  4ad8795ff964a520a21121e3  4bf58dd8d48988d129951735   
6536        280  4a3d735cf964a5204ba21fe3  4bf58dd8d48988d1f6941735   
21636       240  4ab3da04f964a520c36e20e3  4bf58dd8d48988d1c1941735   
140182      236  4e24cec8d4c04ce22533785e  4bf58dd8d48988d1ca941735   
222612      263  4a94318cf964a520b32020e3  4bf58dd8d48988d124941735   
177321       82  4a19b667f964a5203a7a1fe3  4bf58dd8d48988d114951735   
130579

In [8]:
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
df['Venue ID'] = le.fit_transform(df['Venue ID'])
df['Venue category ID'] = le.fit_transform(df['Venue category ID'])
df.drop(['Venue category name'],axis=1,inplace=True)
print(df)

        User ID  Venue ID  Venue category ID   Latitude  Longitude  Timezone  \
42651       667        36                  1  40.927147 -74.118123      -240   
63402       424         8                 26  40.705888 -73.990002      -240   
211582      463        47                  7  40.764031 -73.984036      -300   
171125       25        29                 10  40.761383 -73.971659      -300   
164508      570         4                 17  40.774507 -73.981370      -240   
58374       178        31                  9  40.750801 -73.991299      -240   
182287      806         5                  8  40.722649 -73.986257      -300   
86066       289        21                 11  40.861493 -73.890580      -240   
6536        280        10                 30  40.751559 -73.982638      -240   
21636       240        16                 21  40.731129 -73.993531      -240   
140182      236        35                 24  40.760640 -73.987385      -240   
222612      263        14               

In [ ]:
from datetime import datetime

def str2date(date_str):

    parsed_date = datetime.strptime(date_str, "%a %b %d %H:%M:%S")
    print(parsed_date)

In [9]:
import math

def distance_between_coordinates(lat1, lon1, lat2, lon2):
    radius = 6371  # Radius of the Earth in km

    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Calculate the differences in latitude and longitude<
    d_lat = lat2 - lat1
    d_lon = lon2 - lon1

    # Calculate the distance using the Haversine formula
    a = math.sin(d_lat/2) * math.sin(d_lat/2) + math.cos(lat1) * math.cos(lat2) * math.sin(d_lon/2) * math.sin(d_lon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = radius * c

    return distance

In [ ]:
print(distance_between_coordinates(40.719810,-74.002581,40.735981, -74.029309))

2.8819941850086326


In [10]:
venuesdf = df[['Venue ID','Latitude','Longitude','Venue category ID']].drop_duplicates(subset=['Venue ID'])
print(venuesdf)


        Venue ID   Latitude  Longitude  Venue category ID
42651         36  40.927147 -74.118123                  1
63402          8  40.705888 -73.990002                 26
211582        47  40.764031 -73.984036                  7
171125        29  40.761383 -73.971659                 10
164508         4  40.774507 -73.981370                 17
58374         31  40.750801 -73.991299                  9
182287         5  40.722649 -73.986257                  8
86066         21  40.861493 -73.890580                 11
6536          10  40.751559 -73.982638                 30
21636         16  40.731129 -73.993531                 21
140182        35  40.760640 -73.987385                 24
222612        14  40.725050 -73.996860                 10
177321         7  40.780119 -73.776498                  4
130579        28  40.749631 -73.975436                 10
66770         34  40.813941 -73.989990                 34
51388         46  40.701425 -73.918346                  1
225671        

In [1]:
!pip3 install torch --index-url https://download.pytorch.org/whl/${CUDA}
import torch
print(torch.__version__)
print(torch.version.cuda)
!pip install torch-geometric \
  torch-sparse \
  torch-scatter \
  torch-cluster \
  -f https://pytorch-geometric.com/whl/torch-2.2.0+cu121.html

Looking in indexes: https://download.pytorch.org/whl/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [11]:
import networkx as nx
venuesdict = venuesdf.to_dict('records')

i = 0
nodeIds = {}
node_attr = []

for el in venuesdict:
  #print(el['Venue ID'])
  nodeIds[el['Venue ID']] = i
  node_attr.append(el['Venue category ID'])
  i+=1

print(len(node_attr))


50


In [12]:
#fast spatial graph construction

import networkx as nx
venuesdict = venuesdf.to_dict('records')
#print(venuesdict)

#for el in venuesdict:
#  print((el,venuesdict[el]))
'''
G = nx.Graph()

distances = []

from tqdm import tqdm

for i in tqdm(range(len(venuesdict) - 1)):
    if not G.has_node(venuesdict[i]['Venue ID']):
        G.add_node(venuesdict[i]['Venue ID'], category=venuesdict[i]['Venue category ID'])

    for j in range(i + 1, len(venuesdict)):
        if not G.has_node(venuesdict[j]['Venue ID']):
            G.add_node(venuesdict[j]['Venue ID'], category=venuesdict[i]['Venue category ID'])

        if distance_between_coordinates(venuesdict[i]['Latitude'], venuesdict[i]['Longitude'],
                                        venuesdict[j]['Latitude'], venuesdict[j]['Longitude']) < 1:
            G.add_edge(venuesdict[i]['Venue ID'], venuesdict[j]['Venue ID'])
'''
from tqdm import tqdm
import torch
from torch_geometric.data import Data

edge_index = []
edges_attr = []

for i in tqdm(range(len(venuesdict) - 1)):

   #if venuesdict[i]['Venue ID'] not in node_attr:
   #  node_attr.append(venuesdict[i]['Venue ID'])

   for j in range(i + 1, len(venuesdict)):
    #  if venuesdict[j]['Venue ID'] not in node_attr:
    #     node_attr.append(venuesdict[j]['Venue ID'])


      if distance_between_coordinates(venuesdict[i]['Latitude'], venuesdict[i]['Longitude'],
                                    venuesdict[j]['Latitude'], venuesdict[j]['Longitude']) < 1:
        edge_index.append([nodeIds[venuesdict[i]['Venue ID']], nodeIds[venuesdict[j]['Venue ID']]])
        edges_attr.append(1)

spatG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(spatG)


100%|██████████| 49/49 [00:00<00:00, 6707.82it/s]

Data(x=[50], edge_index=[2, 58], edge_attr=[58])


In [14]:
import pickle

#spatpG = G

pickle.dump(spatG,open('spatG.pickle','wb'))

#data = pickle.load(open('spatG.pickle','rb'))

In [15]:
#nx.draw(G)
print(spatG)

Data(x=[50], edge_index=[2, 58], edge_attr=[58])


In [ ]:
#spatial graph construction

print(len(venuesdf))
import networkx as nx

spatG = nx.Graph()

d = 1

# Add nodes and edges with features
for _, row1 in venuesdf.iterrows():

    if not spatG.has_node(row1['Venue ID']):
            spatG.add_node(row1['Venue ID'], category=row1['Venue category ID'])

    for _,row2 in venuesdf.iterrows():


        # Add nodes if not already in the graph


        if not spatG.has_node(row2['Venue ID']):
            spatG.add_node(row1['Venue ID'], category=row1['Venue category ID'])

        # Add edges with features

        if distance_between_coordinates(row1['Latitude'],row1['Longitude'],row2['Latitude'],row2['Longitude']) < d:

            spatG.add_edge(row1['Venue ID'],row2['Venue ID'])


print(spatG)




13416


KeyboardInterrupt: 

In [ ]:

#sequantial relation graph
#build a graph that captures the sequence of actions made by the users

import networkx as nx

edge_index = []
edges_attr = []

usrloc = {}

#seqG = nx.Graph()

for _, row in df.iterrows():

  #if not seqG.has_node(row['Venue ID']):
  #  seqG.add_node(row['Venue ID'], category=row['Venue category ID'])

  if row['User ID'] not in usrloc:
    usrloc[row['User ID']] = row['Venue ID']

  else:
    #seqG.add_edge(usrloc[row['User ID']],row['Venue ID'])
    edge_index.append([nodeIds[usrloc[row['User ID']]], nodeIds[row['Venue ID']]])
    edges_attr.append(1)
    usrloc[row['User ID']] = row['Venue ID']


seqG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(seqG)






Data(x=[13322], edge_index=[2, 28917], edge_attr=[28917])


In [ ]:
import pickle


print(seqG)

pickle.dump(seqG,open('tempG.pickle','wb'))

#data = pickle.load(open('tempG.pickle','rb'))

Graph with 26132 nodes and 78480 edges


In [ ]:
#build an alternative temporal graph
#two pois are connected if users go on average to them in the seme timeframe

df['localTime'] = pd.to_datetime(df['UTC time']) + pd.to_timedelta(df['Timezone'], unit = 'm')

df['localDayofWeek'] = df.localTime.dt.dayofweek
df['localHour'] = df.localTime.dt.hour






<ipython-input-11-f85843e15d2c>:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['localTime'] = pd.to_datetime(df['UTC time']) + pd.to_timedelta(df['Timezone'], unit = 'm')


In [ ]:
import math
groupeddf = df.groupby(['Venue ID','Venue category ID'],as_index=False)
avghour = groupeddf['localHour'].mean()
avgday = groupeddf['localDayofWeek'].mean()

print(avghour)

timedf = pd.concat([avghour,avgday['localDayofWeek']],axis=1)



print(timedf)

hours = {}
week = {}

#week['workdays'] = []
#week['weekend'] = []


for _, row in timedf.iterrows():

  hour = math.floor(row['localHour'])
  day = math.floor(row['localDayofWeek'])

  if hour not in hours:
    #hours[hour] = [[row['Venue ID'],row['Venue category ID']]]
    hours[hour] = [[row['Venue ID'],row['Venue category ID']]]

  else:
    hours[hour].append([row['Venue ID'],row['Venue category ID']])

  #if 1 <= day <=4:
  #  week['workdays'].append([row['Venue ID'],row['Venue category ID']])
  #else:
  #  week['weekend'].append([row['Venue ID'],row['Venue category ID']])

  if day not in week:
    #hours[hour] = [[row['Venue ID'],row['Venue category ID']]]
    week[day] = [[row['Venue ID'],row['Venue category ID']]]

  else:
    week[day].append([row['Venue ID'],row['Venue category ID']])






       Venue ID  Venue category ID  localHour
0             0                153       13.0
1             1                253       23.0
2             2                 43       18.0
3             2                 56       10.5
4             3                 47       22.0
...         ...                ...        ...
13415     13317                136        2.0
13416     13318                302       16.0
13417     13319                117       17.0
13418     13320                 69        7.0
13419     13321                136        7.0

[13420 rows x 3 columns]
       Venue ID  Venue category ID  localHour  localDayofWeek
0             0                153       13.0             2.0
1             1                253       23.0             5.0
2             2                 43       18.0             4.0
3             2                 56       10.5             5.0
4             3                 47       22.0             4.5
...         ...                ...        ...     

In [ ]:
#dayG = nx.Graph()
#hourG = nx.Graph()

edge_index = []
edges_attr = []

for el in hours:
   for u in hours[el]:

    #if not hourG.has_node(u[0]):
    #  hourG.add_node(u[0], category=u[1])

    for v in hours[el]:

      if u != v:

        #if not hourG.has_node(u[0]):
        #  hourG.add_node(v[0], category=v[1])

        #hourG.add_edge(u[0],v[0])
        edge_index.append([nodeIds[u[0]], nodeIds[v[0]]])

hourG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(hourG)

print(hourG)



Data(x=[13322], edge_index=[2, 10341172], edge_attr=[0])
Data(x=[13322], edge_index=[2, 10341172], edge_attr=[0])


In [ ]:
edge_index = []
edges_attr = []


for el in week:
   for u in week[el]:

    #if not dayG.has_node(u[0]):
    #  dayG.add_node(u[0], category=u[1])

    for v in week[el]:

      if u != v:

        #if not dayG.has_node(u[0]):
        #  dayG.add_node(v[0], category=v[1])

        #dayG.add_edge(u[0],v[0])
        edge_index.append([nodeIds[u[0]], nodeIds[v[0]]])

dayG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(dayG)

print(dayG)


Data(x=[13322], edge_index=[2, 27208004], edge_attr=[0])
Data(x=[13322], edge_index=[2, 27208004], edge_attr=[0])
